In [1]:
# pip install pyfaidx biopython

In [1]:
import pandas as pd
import numpy as np
import torch  # Chuyển từ TF sang Torch
import os
from sklearn.metrics import confusion_matrix
import json
import time
from tqdm import tqdm
from metrics import compute_metrics
from data_preparation import prepare_csv_datasets

# Đường dẫn tới file model SPLAM (thường là splam_script.pt)
MODEL_PATH = r"D:\my_project\Bio_paper\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SPLAM\pretrained_model\splam_script.pt"
PREPARED_FOLDER = "prepared_data/"
RESULTS_DIR = "results/"
BATCH_SIZE = 256  
CONTEXT = 400     
TARGET_LEN = 800
test_files = ['test_1_1_1.csv', 'test_2_1_1.csv', 'test_4_1_1.csv', 'test_10_1_1.csv', 'test_data.csv']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Data preparation

In [3]:
os.makedirs(PREPARED_FOLDER, exist_ok=True)
prepare_csv_datasets(test_files)

[01:30:29] Loading Genome...
🚀 Processing: test_1_1_1.csv (26310 rows)


100%|██████████| 26310/26310 [00:00<00:00, 34708.13it/s]



Type       | Motif at Expected Index   | Status
------------------------------------------------------------
Donor      | Index 200: [GT] | ✅
Donor      | Index 200: [TG] | ❌
Donor      | Index 200: [TG] | ❌
Donor      | Index 200: [TG] | ❌
Donor      | Index 200: [GT] | ✅
Acceptor   | Index 600: [AG] | ✅
Acceptor   | Index 600: [TA] | ❌
Acceptor   | Index 600: [AG] | ✅
Acceptor   | Index 600: [CA] | ❌
Acceptor   | Index 600: [AG] | ✅
✅ Saved to prepared_data/test_1_1_1.csv | Speed: 23026.48 seq/s

🚀 Processing: test_2_1_1.csv (35132 rows)


100%|██████████| 35132/35132 [00:01<00:00, 34624.11it/s]



Type       | Motif at Expected Index   | Status
------------------------------------------------------------
Donor      | Index 200: [TA] | ❌
Donor      | Index 200: [GT] | ✅
Donor      | Index 200: [TC] | ❌
Donor      | Index 200: [GT] | ✅
Donor      | Index 200: [GT] | ✅
Acceptor   | Index 600: [AG] | ✅
Acceptor   | Index 600: [AA] | ❌
Acceptor   | Index 600: [AG] | ✅
Acceptor   | Index 600: [AG] | ✅
Acceptor   | Index 600: [CA] | ❌
✅ Saved to prepared_data/test_2_1_1.csv | Speed: 21916.49 seq/s

🚀 Processing: test_4_1_1.csv (52776 rows)


...


Type       | Motif at Expected Index   | Status
------------------------------------------------------------
Donor      | Index 200: [TG] | ❌
Donor      | Index 200: [TA] | ❌
Donor      | Index 200: [TG] | ❌
Donor      | Index 200: [TG] | ❌
Donor      | Index 200: [GT] | ✅
Acceptor   | Index 600: [AG] | ✅
Acceptor   | Index 600: [TA] | ❌
Acceptor   | Index 600: [CA] | ❌
Acceptor   | Index 600: [AG] | ✅
Acceptor   | Index 600: [AG] | ✅
✅ Saved to prepared_data/test_4_1_1.csv | Speed: 22834.42 seq/s

🚀 Processing: test_10_1_1.csv (105708 rows)


100%|██████████| 105708/105708 [00:02<00:00, 35271.04it/s]



Type       | Motif at Expected Index   | Status
------------------------------------------------------------
Donor      | Index 200: [TA] | ❌
Donor      | Index 200: [GT] | ✅
Donor      | Index 200: [GT] | ✅
Donor      | Index 200: [GT] | ✅
Donor      | Index 200: [TA] | ❌
Acceptor   | Index 600: [CA] | ❌
Acceptor   | Index 600: [TA] | ❌
Acceptor   | Index 600: [CA] | ❌
Acceptor   | Index 600: [AG] | ✅
Acceptor   | Index 600: [AG] | ✅
✅ Saved to prepared_data/test_10_1_1.csv | Speed: 22598.00 seq/s

🚀 Processing: test_data.csv (938297 rows)


100%|██████████| 938297/938297 [00:26<00:00, 35630.49it/s]



Type       | Motif at Expected Index   | Status
------------------------------------------------------------
Donor      | Index 200: [GT] | ✅
Donor      | Index 200: [TA] | ❌
Donor      | Index 200: [TA] | ❌
Donor      | Index 200: [TG] | ❌
Donor      | Index 200: [GT] | ✅
Acceptor   | Index 600: [AG] | ✅
Acceptor   | Index 600: [AG] | ✅
Acceptor   | Index 600: [AG] | ✅
Acceptor   | Index 600: [CA] | ❌
Acceptor   | Index 600: [CA] | ❌
✅ Saved to prepared_data/test_data.csv | Speed: 22851.02 seq/s



In [4]:
df = pd.read_csv(r'D:\my_project\Bio_paper\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SPLAM\prepared_data\test_1_1_1.csv')
# Kiểm tra 5 mẫu Donor đầu tiên
donor_samples = df[df['Splicing_types'] == 1]['sequence'].values[:5]
for i, seq in enumerate(donor_samples):
    # Lấy vùng trung tâm (index 5000 là nucleotide tại vị trí pos)
    center = seq[400:402] 
    print(f"Donor {i}: Vị trí trung tâm là '{center}' (Kỳ vọng thường là GT)")

# Kiểm tra 5 mẫu Acceptor đầu tiên
acceptor_samples = df[df['Splicing_types'] == 2]['sequence'].values[:5]
for i, seq in enumerate(acceptor_samples):
    center = seq[398:400]
    print(f"Acceptor {i}: Vị trí trung tâm là '{center}' (Kỳ vọng thường là AG)")

def diagnose_splice_sites(df, sample_size=5):
    print(f"{'Type':<10} | {'Window (-5 to +5 around center)':<20} | {'Found?'}")
    print("-" * 50)
    
    for label, name in [(1, 'Donor'), (2, 'Acceptor')]:
        samples = df[df['Splicing_types'] == label].sample(min(sample_size, len(df)))
        for _, row in samples.iterrows():
            seq = row['sequence']
            # Lấy 11 nucleotide (vị trí 5000 là trung tâm)
            window = seq[395:406]
            
            target = "GT" if label == 1 else "AG"
            found = "✅" if target in window else "❌"
            
            # Highlight vị trí trung tâm bằng dấu ngoặc []
            display_win = window[:5] + "[" + window[5:7] + "]" + window[7:]
            print(f"{name:<10} | {display_win:<20} | {found} (Target: {target})")

# Chạy sau khi load df từ prepared_data
diagnose_splice_sites(df)

Donor 0: Vị trí trung tâm là 'GT' (Kỳ vọng thường là GT)
Donor 1: Vị trí trung tâm là 'GT' (Kỳ vọng thường là GT)
Donor 2: Vị trí trung tâm là 'GT' (Kỳ vọng thường là GT)
Donor 3: Vị trí trung tâm là 'TA' (Kỳ vọng thường là GT)
Donor 4: Vị trí trung tâm là 'GT' (Kỳ vọng thường là GT)
Acceptor 0: Vị trí trung tâm là 'AG' (Kỳ vọng thường là AG)
Acceptor 1: Vị trí trung tâm là 'AG' (Kỳ vọng thường là AG)
Acceptor 2: Vị trí trung tâm là 'AG' (Kỳ vọng thường là AG)
Acceptor 3: Vị trí trung tâm là 'CA' (Kỳ vọng thường là AG)
Acceptor 4: Vị trí trung tâm là 'AG' (Kỳ vọng thường là AG)
Type       | Window (-5 to +5 around center) | Found?
--------------------------------------------------
Donor      | CACAG[GT]CAGT        | ✅ (Target: GT)
Donor      | GTTGT[GT]AAGT        | ✅ (Target: GT)
Donor      | AAAAG[GT]ACAC        | ✅ (Target: GT)
Donor      | TGAAA[GC]CATT        | ❌ (Target: GT)
Donor      | CTACG[GT]AAGA        | ✅ (Target: GT)
Acceptor   | CAAAA[GC]CTTC        | ✅ (Target: AG)
Acce

### Inference

In [2]:
# Load model SPLAM (TorchScript)
print(f"Loading SPLAM Model on {device}...")
model = torch.jit.load(MODEL_PATH)
model.to(device)
model.eval()
print("Successfully loaded SPLAM model.")

Loading SPLAM Model on cuda...
Successfully loaded SPLAM model.


In [3]:
os.makedirs(RESULTS_DIR, exist_ok=True)

def one_hot_encode_splam(seq):
    """
    Mã hóa chuỗi DNA thành ma trận (4, 800)
    Thứ tự các hàng: 0:A, 1:C, 2:G, 3:T
    """
    seq = seq.upper()
    seq_code = np.zeros((4, len(seq)), dtype=np.float32)
    
    # Sử dụng numpy vectorization để tăng tốc
    sn = np.frombuffer(seq.encode('ascii'), dtype=np.int8)
    seq_code[0, sn == ord('A')] = 1.0
    seq_code[1, sn == ord('C')] = 1.0
    seq_code[2, sn == ord('G')] = 1.0
    seq_code[3, sn == ord('T')] = 1.0
    
    # Mọi ký tự khác (N, R, Y...) sẽ để là vector [0, 0, 0, 0]
    return seq_code

def debug_one_hot_splam(sequences, labels, sample_size=1):
    print("\n--- [CHECKING ALIGNMENT] ---")
    indices = np.random.choice(len(sequences), sample_size)
    for idx in indices:
        seq = sequences[idx]
        label = labels[idx]
        if label == 1: # Donor
            print(f"✅ Mẫu {idx} (Donor): Motif tại 200: {seq[400:402]}")
        elif label == 2: # Acceptor
            print(f"✅ Mẫu {idx} (Acceptor): Motif tại 598:600: {seq[398:400]}")

def get_batches(sequences, batch_size):
    for i in range(0, len(sequences), batch_size):
        batch_seqs = sequences[i:i + batch_size]
        # Encode và stack thành tensor (Batch, 4, 800)
        batch_data = np.array([one_hot_encode_splam(s) for s in batch_seqs])
        yield torch.tensor(batch_data, dtype=torch.float32).to(device)

pbar_files = tqdm(test_files, desc="Overall Progress")

for csv_file in pbar_files:
    path = os.path.join(PREPARED_FOLDER, csv_file)
    if not os.path.exists(path): continue
    
    df = pd.read_csv(path)
    y_true = df['Splicing_types'].values
    debug_one_hot_splam(df['sequence'].values,labels=y_true)
    sequences = df['sequence'].values
    
    all_probs = []
    inference_start = time.time()
    
    with torch.no_grad():
        for batch_tensor in tqdm(get_batches(sequences, BATCH_SIZE), 
                                total=int(np.ceil(len(sequences)/BATCH_SIZE)), 
                                desc=" ↪ Inferencing", leave=False):
            outputs = model(batch_tensor) # (Batch, 3, 800)
        
            # Chuyển Logits sang Probs (Softmax trên chiều lớp dim=1)
            probs = torch.softmax(outputs, dim=1).cpu().numpy()

            # Lấy xác suất của cả 3 lớp tại ĐÚNG vị trí junction (index 400)
            # SPLAM: 0=Null, 1=Acceptor, 2=Donor
            p_null = probs[:, 0, 400]
            p_acc  = probs[:, 1, 400]
            p_don  = probs[:, 2, 400]

            # Stack theo đúng thứ tự nhãn của bạn: 0: Null, 1: Donor, 2: Acceptor
            batch_probs = np.stack([p_null, p_don, p_acc], axis=1)
            all_probs.append(batch_probs)

    # --- SAU VÒNG LẶP ---
    avg_probs = np.vstack(all_probs)
    preds_mapped = np.argmax(avg_probs, axis=1) # Dùng argmax bình thường
    
    total_duration = time.time() - inference_start

    # --- LƯU KẾT QUẢ ---
    try:
        from metrics import compute_metrics 
        # Sử dụng final_probs để tính AUC/AUPRC
        results = compute_metrics(y_true, preds_mapped, probs=avg_probs)
        
        results['benchmarking'] = {
            'total_inference_time_sec': round(total_duration, 4),
            'samples_per_second': round(len(df) / total_duration, 2)
        }
        
        cm = confusion_matrix(y_true, preds_mapped)
        output = {"metrics": results, "confusion_matrix": cm.tolist()}
        
        json_path = os.path.join(RESULTS_DIR, csv_file.replace('.csv', '_results.json'))
        with open(json_path, 'w') as f:
            json.dump(output, f, indent=4)
            
        print(f"✅ {csv_file} processed. Speed: {results['benchmarking']['samples_per_second']} seq/s")
            
    except Exception as e:
        print(f"\n❌ Error with {csv_file}: {e}")

print("\n✨ SPLAM INFERENCE COMPLETED!")

Overall Progress:   0%|          | 0/5 [00:00<?, ?it/s]


--- [CHECKING ALIGNMENT] ---


Overall Progress:  20%|██        | 1/5 [00:15<01:03, 15.94s/it]

✅ test_1_1_1.csv processed. Speed: 1669.35 seq/s

--- [CHECKING ALIGNMENT] ---
✅ Mẫu 14288 (Donor): Motif tại 200: TA


Overall Progress:  40%|████      | 2/5 [00:35<00:54, 18.24s/it]

✅ test_2_1_1.csv processed. Speed: 1788.54 seq/s

--- [CHECKING ALIGNMENT] ---


Overall Progress:  60%|██████    | 3/5 [01:03<00:45, 22.64s/it]

✅ test_4_1_1.csv processed. Speed: 1917.52 seq/s

--- [CHECKING ALIGNMENT] ---


Overall Progress:  60%|██████    | 3/5 [01:07<00:44, 22.43s/it]


KeyboardInterrupt: 

In [ ]:
# Chạy thử cho 5 mẫu Donor (y_true == 1)
test_idx = df[df['Splicing_types'] == 1].index[:5]
for idx in test_idx:
    seq = df.iloc[idx]['sequence']
    input_t = torch.tensor(one_hot_encode_splam(seq)).unsqueeze(0).to(device)
    out = torch.softmax(model(input_t), dim=1)[0].cpu().detach().numpy()
    
    print(f"--- Mẫu Donor {idx} ---")
    print(f"Lớp 1 (Acc) tại 200: {out[1, 200]:.4f} | Lớp 2 (Don) tại 200: {out[2, 200]:.4f}")
    print(f"Lớp 1 (Acc) tại 600: {out[1, 600]:.4f} | Lớp 2 (Don) tại 600: {out[2, 600]:.4f}")

--- Mẫu Donor 0 ---
Lớp 1 (Acc) tại 200: 0.2119 | Lớp 2 (Don) tại 200: 0.2120
Lớp 1 (Acc) tại 600: 0.2121 | Lớp 2 (Don) tại 600: 0.2120
--- Mẫu Donor 5 ---
Lớp 1 (Acc) tại 200: 0.2122 | Lớp 2 (Don) tại 200: 0.2128
Lớp 1 (Acc) tại 600: 0.2123 | Lớp 2 (Don) tại 600: 0.2120
--- Mẫu Donor 9 ---
Lớp 1 (Acc) tại 200: 0.2119 | Lớp 2 (Don) tại 200: 0.2119
Lớp 1 (Acc) tại 600: 0.2119 | Lớp 2 (Don) tại 600: 0.2119
--- Mẫu Donor 15 ---
Lớp 1 (Acc) tại 200: 0.2119 | Lớp 2 (Don) tại 200: 0.2119
Lớp 1 (Acc) tại 600: 0.2119 | Lớp 2 (Don) tại 600: 0.2119
--- Mẫu Donor 16 ---
Lớp 1 (Acc) tại 200: 0.2119 | Lớp 2 (Don) tại 200: 0.2119
Lớp 1 (Acc) tại 600: 0.2119 | Lớp 2 (Don) tại 600: 0.2119


In [ ]:
# Chạy đoạn này trong Notebook của bạn
with torch.no_grad():
    sample_don = torch.tensor(one_hot_encode_splam(donor_sequence)).unsqueeze(0).to(device)
    output = torch.softmax(model(sample_don), dim=1)[0].cpu().numpy() # (3, 800)
    
    # Tìm vị trí mà lớp Donor (2) đạt Max
    peak_pos = np.argmax(output[2, :])
    print(f"Donor Peak tìm thấy tại index: {peak_pos} với xác suất {output[2, peak_pos]:.4f}")